<a href="https://colab.research.google.com/github/sbuttler/ddnv/blob/main/Hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install scikit-learn==0.23.1 tensorflow==2.1.0  keras==2.3.1 ddop==0.6.7

In [1]:
from sklearn.utils.validation import check_array
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import make_scorer
from ddop.datasets import load_yaz, load_bakery
from ddop.metrics import average_costs, prescriptiveness_score
from ddop.newsvendor import SampleAverageApproximationNewsvendor
from ddop.newsvendor import DecisionTreeWeightedNewsvendor
from ddop.newsvendor import RandomForestWeightedNewsvendor 
from ddop.newsvendor import KNeighborsWeightedNewsvendor
from ddop.newsvendor import LinearRegressionNewsvendor
from ddop.newsvendor import GaussianWeightedNewsvendor
from ddop.newsvendor import LinearRegressionNewsvendor
from ddop.newsvendor import DeepLearningNewsvendor
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import statistics

In [ ]:
import logging

In [2]:
n_features = 12

In [3]:
np.arange(1,11)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [4]:
#TODO: Set Grids
dl = {"optimizer": ["adam"],
      "neurons": [
                  (round(0.5*n_features),round(0.5*0.5*n_features)),
                  (round(0.5*n_features),round(0.5*1*n_features)),
                  (1*n_features,round(1*0.5*n_features)),
                  (1*n_features,1*1*n_features),
                  (2*n_features,round(2*0.5*n_features)),
                  (2*n_features,2*1*n_features),
                  (3*n_features,round(3*0.5*n_features)),
                  (3*n_features,3*1*n_features)],
      "epochs": [10,100,200]}

dtw = {"max_depth":[None,2,3,4,5,6,7,8,9,10,15]}

rfw = {"max_depth":[None,2,4,6,8,10,15],
          'min_samples_split':[2,4,6,8,10,50],
          'n_estimators':[2,10,20,50,100]}

knnw = {'n_neighbors':[1,2,4,8,16,32,64,128]}

gkw = {'kernel_bandwidth':[1,1.2,1.4,1.6,1.8,2,2.5,3]}

In [5]:
# Define model tuples: 'model_name', model, grid
estimator_tuple_list = []
#estimator_tuple_list.append(('SAA', SampleAverageApproximationNewsvendor(),None))
#estimator_tuple_list.append(('DTW', DecisionTreeWeightedNewsvendor(random_state=1),dtw))
estimator_tuple_list.append(('RFW', RandomForestWeightedNewsvendor(n_jobs=2, random_state=1),rfw))
#estimator_tuple_list.append(('KNNW',KNeighborsWeightedNewsvendor(),knnw))
#estimator_tuple_list.append(('GKW', GaussianWeightedNewsvendor(),gkw))
#estimator_tuple_list.append(('DL', DeepLearningNewsvendor(),[dl]))
#estimator_tuple_list.append(('LR', LinearRegressionNewsvendor(),None))

--------------------------------------
# YAZ Dataset

In [57]:
data = load_yaz(one_hot_encoding=True)
X = data.data
y = data.target

In [58]:
products = y.columns.to_list()

With target scaling and 10 fold CV

In [59]:
estimators = []
results_sc = pd.DataFrame()
for cu, co in zip([5,7.5,9],[5,2.5,1]):
#for cu, co in zip([7.5],[2.5]):
  for  estimator_tuple in estimator_tuple_list:
    costs = []
    score = []
    for product in products:
      
      X_train, X_test, y_train, y_test = train_test_split(X, y[product], train_size=0.75, shuffle=False)
      scaler = StandardScaler()
      scaler.fit(X_train)
      X_train = scaler.transform(X_train)
      X_test = scaler.transform(X_test)
      
      #scale target variable
      scaler_target = StandardScaler()
      scaler_target.fit(np.array(y_train).reshape(-1, 1))
      y_train = scaler_target.transform(np.array(y_train).reshape(-1, 1))
      #y_test = scaler_target.transform(np.array(y_test).reshape(-1, 1))

      saa_pred = SampleAverageApproximationNewsvendor(cu,co).fit(y_train).predict(y_test.shape[0])
      saa_pred = scaler_target.inverse_transform(saa_pred)
      estimator_name = estimator_tuple[0]
      param_grid = estimator_tuple[2]
      estimator = estimator_tuple[1]
      estimator.set_params(cu=cu,co=co)
      print(estimator)

      if param_grid == None:
        if estimator_name=="SAA":
          pred = estimator.fit(y_train).predict(X_test.shape[0])
          pred = scaler_target.inverse_transform(pred)
        else:
          pred = estimator.fit(X_train,y_train).predict(X_test)
          pred = scaler_target.inverse_transform(pred)
        
      else:
        cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
        gs = GridSearchCV(estimator, param_grid, cv=cv, n_jobs=-1)
        gs.fit(X_train,y_train)
        best_estimator = gs.best_estimator_
        pred = best_estimator.predict(X_test)
        pred = scaler_target.inverse_transform(pred)
        estimators.append(best_estimator)

      avg_cost = average_costs(y_test,pred,cu,co,multioutput="uniform_average")
      p_score = prescriptiveness_score(y_test, pred, saa_pred, cu, co, multioutput="uniform_average")
      costs.append(avg_cost)
      score.append(p_score)

    d = {'SL': [cu/(cu+co)], 'Model': [estimator_name]}  
    for i in range(len(costs)):
      d[products[i]+" AC"] = costs[i]
    for i in range(len(costs)):
      d[products[i]+" SoP"] = score[i]

    average_cost = statistics.mean(costs)
    d["Average Cost"] = average_cost
    presc_score = statistics.mean(score)
    d["Score of Prescriptiveness"] = presc_score
    df = pd.DataFrame(data=d)
    results_sc = pd.concat([results_sc,df])

RandomForestWeightedNewsvendor(co=5, cu=5, random_state=1)


/media/data/anaconda3/envs/ddnv_env/lib/python3.7/site-packages/ddop/newsvendor/_WeightedNewsvendor.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.model_ = model.fit(X, y)


RandomForestWeightedNewsvendor(co=5, cu=5, random_state=1)


/media/data/anaconda3/envs/ddnv_env/lib/python3.7/site-packages/ddop/newsvendor/_WeightedNewsvendor.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.model_ = model.fit(X, y)


RandomForestWeightedNewsvendor(co=5, cu=5, random_state=1)


...

/media/data/anaconda3/envs/ddnv_env/lib/python3.7/site-packages/ddop/newsvendor/_WeightedNewsvendor.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.model_ = model.fit(X, y)


RandomForestWeightedNewsvendor(co=1, cu=9, random_state=1)


/media/data/anaconda3/envs/ddnv_env/lib/python3.7/site-packages/ddop/newsvendor/_WeightedNewsvendor.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.model_ = model.fit(X, y)


RandomForestWeightedNewsvendor(co=1, cu=9, random_state=1)


/media/data/anaconda3/envs/ddnv_env/lib/python3.7/site-packages/ddop/newsvendor/_WeightedNewsvendor.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.model_ = model.fit(X, y)


In [60]:
results_sc

,SL,Model,calamari AC,fish AC,shrimp AC,chicken AC,koefte AC,lamb AC,steak AC,calamari SoP,fish SoP,shrimp SoP,chicken SoP,koefte SoP,lamb SoP,steak SoP,Average Cost,Score of Prescriptiveness
0,0.50,RFW,8.645833,8.619792,16.093750,38.802083,34.661458,42.109375,28.802083,0.023529,0.034985,0.085799,0.091463,0.082702,0.113487,0.099349,25.390625,0.075902
0,0.75,RFW,7.552083,7.955729,13.125000,33.593750,29.973958,32.395833,24.322917,0.070513,-0.006590,0.126516,0.120654,0.066883,0.195343,0.188532,21.274182,0.108836
0,0.90,RFW,4.354167,5.072917,7.671875,20.734375,16.984375,18.375000,14.588542,0.158954,0.115350,0.098531,0.160835,0.100414,0.160400,0.256240,12.540179,0.150103


In [73]:
grid_values = [] 
for est in estimators:
    
    values = [est.get_params().get(x) for x in ['max_depth', 'min_samples_split', 'n_estimators']]
    
    grid_values.append(values)

grid_v = pd.DataFrame(grid_values, columns = ['max_depth', 'min_samples_split', 'n_estimators'])

In [74]:
grid_v

,max_depth,min_samples_split,n_estimators
0,3.0,8,100
1,4.0,2,100
2,15.0,50,50
3,7.0,4,100
4,8.0,6,100
5,6.0,4,50
6,9.0,6,100
7,3.0,2,50
8,5.0,4,100
9,NaN,50,100


In [ ]:
results.to_excel("results_yaz_lag_per_product.xlsx")

In [ ]:
from google.colab import files

In [ ]:
files.download('results_yaz_lag_per_product.xlsx') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

-----------------------------------------------------------------
# Global Model (ausgegliedert)

In [ ]:
estimators = []
results = pd.DataFrame()
for cu, co in zip([5, 7.5, 9],[5, 2.5, 1]):
  saa_pred = SampleAverageApproximationNewsvendor(cu,co).fit(y_train).predict(y_test.shape[0])
  for estimator_tuple in estimator_tuple_list:
    estimator_name = estimator_tuple[0]
    param_grid = estimator_tuple[2]
    estimator = estimator_tuple[1]
    estimator.set_params(cu=cu,co=co)
    print(estimator)
    if param_grid == None:
      if estimator_name=="SAA":
        pred = estimator.fit(y_train).predict(X_test.shape[0])
      else:
        pred = estimator.fit(X_train,y_train).predict(X_test)
      
    else:
      gs = GridSearchCV(estimator, param_grid)
      gs.fit(X_train,y_train)
      best_estimator = gs.best_estimator_
      pred = best_estimator.predict(X_test)
      estimators.append(best_estimator)
      
    d = {'SL': [cu/(cu+co)], 'Model': [estimator_name]}  
    avg_cost = average_costs(y_test,pred,cu,co, multioutput="raw_values")
    p_score = prescriptiveness_score(y_test, pred, saa_pred, cu, co, multioutput="raw_values")
    for i in range(avg_cost.size):
      d[products[i]+" AC"] = avg_cost[i]
    for i in range(avg_cost.size):
      d[products[i]+" SoP"] = p_score[i]
    avg_cost = average_costs(y_test,pred,cu,co, multioutput="uniform_average")
    d["Average Cost"] = avg_cost
    p_score = prescriptiveness_score(y_test, pred, saa_pred, cu, co, multioutput="uniform_average")
    d["Score of Prescriptiveness"] = p_score
    df = pd.DataFrame(data=d)
    results = pd.concat([results,df])


-----------------------------------------
# Bakery Dataset

In [6]:
bakery = load_bakery(one_hot_encoding=True)
X = bakery.data
y = bakery.target

In [7]:
X_grouped = X.groupby(['product', 'store'])

In [8]:
groups = list(X_grouped.groups.keys())

In [9]:
import time

In [ ]:
import statistics
estimators = []
results = pd.DataFrame()
#for cu, co in zip([5,7.5,9],[5,2.5,1]):
for cu, co in zip([7.5],[2.5]):
  for  estimator_tuple in estimator_tuple_list:
    costs = []
    score = []
    for group in groups:

      X_temp = X_grouped.get_group(group)
      y_temp = y.iloc[X_temp.index.values.tolist()]

      X_train, X_test, y_train, y_test = train_test_split(X_temp, y_temp, train_size=0.75, shuffle=False)
      scaler = StandardScaler()
      scaler.fit(X_train)
      X_train = scaler.transform(X_train)
      X_test = scaler.transform(X_test)

      #scale target variable
      scaler_target = StandardScaler()
      scaler_target.fit(np.array(y_train).reshape(-1, 1))
      y_train = scaler_target.transform(np.array(y_train).reshape(-1, 1))

      saa_pred = SampleAverageApproximationNewsvendor(cu,co).fit(y_train).predict(y_test.shape[0])
      saa_pred = scaler_target.inverse_transform(saa_pred)
      estimator_name = estimator_tuple[0]
      param_grid = estimator_tuple[2]
      estimator = estimator_tuple[1]
      estimator.set_params(cu=cu,co=co)
      print(estimator)

      if param_grid == None:
        if estimator_name=="SAA":
          pred = estimator.fit(y_train).predict(X_test.shape[0])
          pred = scaler_target.inverse_transform(pred)
        else:
          pred = estimator.fit(X_train,y_train).predict(X_test)
          pred = scaler_target.inverse_transform(pred)
        
      else:
        start = time.time()
        print(start)
        cv = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1)
        gs = GridSearchCV(estimator, param_grid, cv=cv, n_jobs=-1)
        gs.fit(X_train,y_train)
        best_estimator = gs.best_estimator_
        pred = best_estimator.predict(X_test)
        pred = scaler_target.inverse_transform(pred)
        estimators.append(best_estimator)
        print(time.time() - start)

      avg_cost = average_costs(y_test,pred,cu,co,multioutput="uniform_average")
      p_score = prescriptiveness_score(y_test, pred, saa_pred, cu, co, multioutput="uniform_average")
      costs.append(avg_cost)
      score.append(p_score)

    d = {'SL': [cu/(cu+co)], 'Model': [estimator_name]}  
    for i in range(len(costs)):
      d[str(groups[i])+" AC"] = costs[i]
    for i in range(len(costs)):
      d[str(groups[i])+" SoP"] = score[i]

    average_cost = statistics.mean(costs)
    d["Average Cost"] = average_cost
    presc_score = statistics.mean(score)
    d["Score of Prescriptiveness"] = presc_score
    df = pd.DataFrame(data=d)
    results = pd.concat([results,df])

RandomForestWeightedNewsvendor(co=2.5, cu=7.5, n_jobs=2, random_state=1)
1623932693.4118004


/media/data/anaconda3/envs/ddnv_env/lib/python3.7/site-packages/ddop/newsvendor/_WeightedNewsvendor.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.model_ = model.fit(X, y)


1543.0233602523804
RandomForestWeightedNewsvendor(co=2.5, cu=7.5, n_jobs=2, random_state=1)
1623934236.44368


In [ ]:
results

In [ ]:
grid_values = [] 
for est in estimators:
    
    values = [est.get_params().get(x) for x in ['max_depth', 'min_samples_split', 'n_estimators']]
    
    grid_values.append(values)

grid_v = pd.DataFrame(grid_values, columns = ['max_depth', 'min_samples_split', 'n_estimators'])

In [ ]:
grid_v.to_csv('grid_results_bakery_rf')